### Grammy 

In [1]:
import os
import pandas as pd
import psycopg2
from dotenv import load_dotenv

### Conexion a la DB 

In [2]:
load_dotenv()

# Crear conexión a la base de datos
try:
    connection = psycopg2.connect(
        dbname=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        host=os.getenv('DB_HOST'),
        port=os.getenv('DB_PORT')
    )
    print("Conexión exitosa a la base de datos PostgreSQL")
except (Exception, psycopg2.Error) as error:
    print("Error al conectar a la base de datos PostgreSQL:", error)

Conexión exitosa a la base de datos PostgreSQL


In [3]:
query = "SELECT * FROM public.grammy;"

try:
    df = pd.read_sql_query(query, connection)
    print("Datos de la tabla leídos correctamente en un DataFrame de Pandas")
except Exception as e:
    print("Error al leer datos de la tabla:", e)

# Cerrar la conexión a la base de datos
if connection is not None:
    connection.close()
    print("Conexión cerrada")

Datos de la tabla leídos correctamente en un DataFrame de Pandas
Conexión cerrada


C:\Users\Mariana Mera\AppData\Local\Temp\ipykernel_12500\3470707302.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


### Analisis del df 

In [4]:
df.head()

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4810 entries, 0 to 4809
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          4810 non-null   int64 
 1   title         4810 non-null   object
 2   published_at  4810 non-null   object
 3   updated_at    4810 non-null   object
 4   category      4810 non-null   object
 5   nominee       4804 non-null   object
 6   artist        2970 non-null   object
 7   workers       2620 non-null   object
 8   img           3443 non-null   object
 9   winner        4810 non-null   bool  
dtypes: bool(1), int64(1), object(8)
memory usage: 343.0+ KB


In [6]:
df.shape

(4810, 10)

### Verificar valores null

In [7]:
df.isnull().sum()

year               0
title              0
published_at       0
updated_at         0
category           0
nominee            6
artist          1840
workers         2190
img             1367
winner             0
dtype: int64

In [8]:
df.nunique()

year              62
title             62
published_at       4
updated_at        10
category         638
nominee         4131
artist          1658
workers         2366
img             1463
winner             1
dtype: int64

### Artist

In [9]:
df["artist"].unique()

array(['Billie Eilish', 'Bon Iver', 'Ariana Grande', ..., 'The Champs',
       'Stan Freberg',
       'David Seville And The Chipmunks (Ross Bagdasarian, Sr.)'],
      dtype=object)

In [10]:
df['workers'].unique()

array(["Finneas O'Connell, producer; Rob Kinelski & Finneas O'Connell, engineers/mixers; John Greenham, mastering engineer",
       'BJ Burton, Brad Cook, Chris Messina & Justin Vernon, producers; BJ Burton, Zach Hanson & Chris Messina, engineers/mixers; Greg Calbi, mastering engineer',
       'Charles Anderson, Tommy Brown, Michael Foster & Victoria Monet, producers; Serban Ghenea, John Hanes, Billy Hickey & Brendan Morawski, engineers/mixers; Randy Merrill, mastering engineer',
       ..., 'Andres Segovia, artist',
       'Hollywood String Quartet (Alvin Dinkin, Paul Shure, Eleanor Aller Slatkin & Felix Slatkin), artists',
       'Roger Wagner, choir director'], dtype=object)

### Verificar si se pueden poner la info de workers en artist 

In [11]:
df.loc[df['artist'].isnull() & ~df['workers'].isnull(), ['artist', 'workers']]

,artist,workers
16,None,"Billie Eilish O'Connell & Finneas O'Connell, s..."
17,None,"Natalie Hemby, Lady Gaga, Hillary Lindsey & Lo..."
18,None,"Brandi Carlile, Phil Hanseroth, Tim Hanseroth ..."
19,None,"Ruby Amanfu, Sam Ashworth, D. Arcelious Harris..."
20,None,"Taylor Swift, songwriter (Taylor Swift)"
...,...,...
4804,None,(Hollywood Bowl Orchestra)
4805,None,"Van Cliburn, artist (Symphony Of The Air Orche..."
4806,None,"Andres Segovia, artist"
4807,None,"Hollywood String Quartet (Alvin Dinkin, Paul S..."


In [12]:
import re

# Crear una expresión regular para encontrar el contenido dentro de los paréntesis
patron = r'\((.*?)\)'

# Seleccionar solo las filas donde la columna 'workers' no es nula
workers_not_null = df['workers'].notnull()

df.loc[workers_not_null, 'workers'].str.extract(patron)

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
4804,Hollywood Bowl Orchestra
4805,Symphony Of The Air Orchestra
4806,NaN
4807,"Alvin Dinkin, Paul Shure, Eleanor Aller Slatki..."


In [13]:
import re

# Crear una expresión regular para encontrar el contenido dentro de los paréntesis
patron = r'\(([^)]+)\)'

# Extraer el contenido entre paréntesis en la columna 'workers'
contenido_entre_parentesis = df['workers'].str.extract(patron)

# Reemplazar los valores NaN en la columna 'artist' con el contenido entre paréntesis
df['artist'] = df['artist'].fillna(contenido_entre_parentesis[0])


In [14]:
df.isnull().sum()

year               0
title              0
published_at       0
updated_at         0
category           0
nominee            6
artist           474
workers         2190
img             1367
winner             0
dtype: int64

In [15]:
# Imputación basada en otra columna
df['artist'].fillna(df['workers'], inplace=True)  # Por ejemplo, utilizando el nombre del nominado


C:\Users\Mariana Mera\AppData\Local\Temp\ipykernel_12500\1209832427.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['artist'].fillna(df['workers'], inplace=True)  # Por ejemplo, utilizando el nombre del nominado


In [16]:
df['workers'].fillna(df['artist'], inplace=True)

C:\Users\Mariana Mera\AppData\Local\Temp\ipykernel_12500\1984585983.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['workers'].fillna(df['artist'], inplace=True)


In [17]:
df.isnull().sum()

year               0
title              0
published_at       0
updated_at         0
category           0
nominee            6
artist           186
workers          186
img             1367
winner             0
dtype: int64

In [18]:
df[df['artist'].isnull() & df['workers'].isnull()].head(5)

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
24,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Billie Eilish,None,None,None,True
25,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Black Pumas,None,None,None,True
26,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Lil Nas X,None,None,None,True
27,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Lizzo,None,None,None,True
28,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Maggie Rogers,None,None,None,True


### Category 

In [19]:
cantidad_total_categorias = df['category'].nunique()
print("Cantidad total de categorías:", cantidad_total_categorias)

Cantidad total de categorías: 638


In [20]:
df2 = df.loc[df['artist'].isnull()]
df2['category'].unique()

array(['Best New Artist', 'Producer Of The Year, Non-Classical',
       'Producer Of The Year, Classical',
       'Remixer of the Year, Non-Classical',
       'Remixer Of The Year, Non-Classical',
       'Best Small Ensemble Performance (With Or Without Conductor)',
       'Remixer Of The Year, Non-classical',
       'Best Classical Vocal Performance', 'Producer Of The Year',
       'Classical Producer Of The Year',
       'Producer Of The Year (Non-Classical)',
       'Producer Of The Year (Non Classical)',
       'Best Classical Vocal Soloist',
       'Producer Of The Year, (Non Classical)',
       'Best Classical Vocal Soloist Performance',
       'Best Classical Performance - Instrumental Soloist Or Soloists (With Or Without Orchestra)',
       'Best New Classical Artist', 'Best New Artist Of The Year',
       'Best Producer Of The Year',
       'Best Performance - Instrumental Soloist Or Soloists (With Or Without Orchestra)',
       'Best New Country & Western Artist',
       'Mos

### Limpieza de categoria 

In [21]:
df['category'] = df['category'].str.replace(r'[-()]', '', regex=True)

# Mostrar las categorías únicas después de la limpieza
categorias_unicas_limpias = df['category'].unique()
print(categorias_unicas_limpias)

['Record Of The Year' 'Album Of The Year' 'Song Of The Year'
 'Best New Artist' 'Best Pop Solo Performance'
 'Best Pop Duo/Group Performance' 'Best Traditional Pop Vocal Album'
 'Best Contemporary Folk Recording' 'Best Pop Vocal Album'
 'Best Dance Recording' 'Best Dance/Electronic Album'
 'Best Contemporary Instrumental Album' 'Best Rock Performance'
 'Best Metal Performance' 'Best Rock Song' 'Best Rock Album'
 'Best Alternative Music Album' 'Best R&B Performance'
 'Best Traditional R&B Performance' 'Best R&B Song'
 'Best Urban Contemporary Album' 'Best R&B Album' 'Best Rap Performance'
 'Best Rap/Sung Performance' 'Best Rap Song' 'Best Rap Album'
 'Best Country Solo Performance' 'Best Country Duo/Group Performance'
 'Best Country Song' 'Best Country Album' 'Best New Age Album'
 'Best Improvised Jazz Solo' 'Best Jazz Vocal Album'
 'Best Jazz Instrumental Album' 'Best Large Jazz Ensemble Album'
 'Best Latin Jazz Album' 'Best Gospel Performance/Song'
 'Best Contemporary Christian Music 

In [22]:
df['category'].value_counts()

category
Song Of The Year                                                            70
Record Of The Year                                                          69
Album Of The Year                                                           66
Best Opera Recording                                                        64
Best Album Notes                                                            63
                                                                            ..
Best Contemporary Pop Performance, Instrumental                              1
Best Rhythm & Blues Vocal Performance, Female                                1
Best Rhythm & Blues Vocal Performance, Male                                  1
Best Rhythm & Blues Performance By A Duo Or Group, Vocal Or Instrumental     1
Best Classical Performance  Operatic Or Choral                               1
Name: count, Length: 630, dtype: int64

In [23]:
df[df['category'].str.contains('Artist', case=False)].head(5)

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
24,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Billie Eilish,None,None,None,True
25,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Black Pumas,None,None,None,True
26,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Lil Nas X,None,None,None,True
27,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Lizzo,None,None,None,True
28,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Maggie Rogers,None,None,None,True


### Reemplazar valores null de Artist por Nominee donde la categoria sean artist 

In [24]:
categorias_artist = df[df['category'].str.contains('Artist', case=False)]

# Asignar los valores de la columna 'nominee' a las columnas 'artist' y 'workers'
df.loc[categorias_artist.index, 'artist'] = categorias_artist['nominee']
df.loc[categorias_artist.index, 'workers'] = categorias_artist['nominee']


In [25]:
df.isnull().sum()

year               0
title              0
published_at       0
updated_at         0
category           0
nominee            6
artist           117
workers          117
img             1367
winner             0
dtype: int64

In [26]:
df[df['artist'].isnull() & df['workers'].isnull()].head(5)

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
505,2018,61st Annual GRAMMY Awards (2018),2018-12-06T23:48:49-08:00,2019-09-10T01:06:11-07:00,"Producer Of The Year, NonClassical",Pharrell Williams,None,None,None,True
509,2018,61st Annual GRAMMY Awards (2018),2018-12-06T23:48:49-08:00,2019-09-10T01:06:11-07:00,"Producer Of The Year, Classical",Blanton Alspaugh,None,None,None,True
592,2017,60th Annual GRAMMY Awards (2017),2018-05-22T03:08:24-07:00,2019-09-10T01:06:11-07:00,"Producer Of The Year, NonClassical",Greg Kurstin,None,None,None,True
596,2017,60th Annual GRAMMY Awards (2017),2018-05-22T03:08:24-07:00,2019-09-10T01:06:11-07:00,"Producer Of The Year, Classical",David Frost,None,None,None,True
676,2016,59th Annual GRAMMY Awards (2016),2017-11-28T00:03:45-08:00,2019-09-10T01:06:11-07:00,"Producer Of The Year, NonClassical",Greg Kurstin,None,None,None,True


In [27]:
# Filtrar las filas donde 'nominee'  es nulo
df[df['nominee'].isnull()]

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
2269,2000,43rd Annual GRAMMY Awards (2000),2017-11-28T00:03:45-08:00,2019-09-10T01:11:09-07:00,"Remixer of the Year, NonClassical",None,None,None,None,True
2367,1999,42nd Annual GRAMMY Awards (1999),2017-11-28T00:03:45-08:00,2019-09-10T01:09:02-07:00,"Remixer Of The Year, NonClassical",None,None,None,None,True
2462,1998,41st Annual GRAMMY Awards (1998),2017-11-28T00:03:45-08:00,2019-09-10T01:08:19-07:00,"Remixer Of The Year, Nonclassical",None,None,None,None,True
2555,1997,40th Annual GRAMMY Awards (1997),2017-11-28T00:03:45-08:00,2019-09-10T01:07:37-07:00,"Remixer Of The Year, NonClassical",None,None,None,None,True
4525,1965,8th Annual GRAMMY Awards (1965),2017-11-28T00:03:45-08:00,2019-09-10T01:06:59-07:00,Best New Country & Western Artist,None,None,None,None,True
4573,1964,7th Annual GRAMMY Awards (1964),2017-11-28T00:03:45-08:00,2019-09-10T01:06:11-07:00,Best New Country & Western Artist Of 1964,None,None,None,None,True


### Elimminar 6 nominee null 

In [28]:
indices_a_eliminar = [2261, 2359, 2454, 2547, 4525, 4573]

# Eliminar las filas
df.drop(indices_a_eliminar, inplace=True)

In [29]:
df.isnull().sum()

year               0
title              0
published_at       0
updated_at         0
category           0
nominee            4
artist           115
workers          115
img             1362
winner             0
dtype: int64

In [30]:
df.isnull().sum()

year               0
title              0
published_at       0
updated_at         0
category           0
nominee            4
artist           115
workers          115
img             1362
winner             0
dtype: int64

In [31]:
columnas_a_eliminar = ['published_at', 'updated_at', 'img']

# Eliminar las columnas en una sola línea
df = df.drop(columns=columnas_a_eliminar)

In [32]:
df.columns

Index(['year', 'title', 'category', 'nominee', 'artist', 'workers', 'winner'], dtype='object')

In [33]:
df.to_csv('grammynew.csv', index=False)